# 📌 Sales Forecasting Using Time Series
## **Project Overview**
This notebook demonstrates how to analyze historical sales data and build a forecasting model using **Prophet** and **SARIMA**.
We will:
- Perform **Exploratory Data Analysis (EDA)**
- Implement **Prophet (by Facebook)** and **SARIMA** models
- Evaluate model performance using **MAE** and **RMSE**

In [ ]:
# 📌 Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## 📌 Step 2: Load the Dataset

In [ ]:
# Load dataset
df = pd.read_csv('data/train.csv', parse_dates=['date'])
df = df[['date', 'store_nbr', 'family', 'sales']]
df = df.sort_values(by=['date'])
df.head()

## 📌 Step 3: Exploratory Data Analysis (EDA)

In [ ]:
# Overall Sales Trend
plt.figure(figsize=(12, 6))
sns.lineplot(x=df['date'], y=df['sales'])
plt.title('Overall Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

## 📌 Step 4: Data Preprocessing

In [ ]:
# Split dataset into training and testing sets
split_date = '2017-06-01'
train = df[df['date'] < split_date]
test = df[df['date'] >= split_date]

## 📌 Step 5: Build and Train Forecasting Models

In [ ]:
# Prophet Model
train_prophet = train[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
model = Prophet()
model.fit(train_prophet)
future = model.make_future_dataframe(periods=len(test))
forecast = model.predict(future)
model.plot(forecast)
plt.show()

In [ ]:
# SARIMA Model
sarima_model = SARIMAX(train['sales'], order=(1,1,1), seasonal_order=(1,1,1,12))
sarima_result = sarima_model.fit()
sarima_forecast = sarima_result.predict(start=len(train), end=len(train) + len(test) - 1)
plt.plot(train['date'], train['sales'], label='Training Data')
plt.plot(test['date'], test['sales'], label='Actual Sales', color='orange')
plt.plot(test['date'], sarima_forecast, label='SARIMA Forecast', color='red')
plt.legend()
plt.show()

## 📌 Step 6: Model Evaluation

In [ ]:
# Define evaluation function
def evaluate_model(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return {'MAE': mae, 'RMSE': rmse}

# Evaluate Prophet
prophet_results = evaluate_model(test['sales'], forecast['yhat'][-len(test):])
# Evaluate SARIMA
sarima_results = evaluate_model(test['sales'], sarima_forecast)

print('Prophet Model Performance:', prophet_results)
print('SARIMA Model Performance:', sarima_results)

## 📌 Step 7: Conclusion
- If **Prophet has lower errors**, it is better for long-term predictions.
- If **SARIMA has lower errors**, it is better for short-term forecasts.
- Businesses can use these models for **inventory planning, revenue predictions, and promotions.**